In [1]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "EleutherAI/pythia-160m-deduped"
dataset_name = "EleutherAI/the_pile_deduplicated"

lm = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
lm.to(device)

dataset = load_dataset(dataset_name, split='train', streaming=True)

/workspace/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [11]:
import json

from safetensors import safe_open

PATH = "../saes/pythia_160m-jr/baseline/8/"
state_dict = safe_open(PATH + "sae.safetensors", framework="torch")
with open(PATH + "cfg.json", "r") as f:
    cfg = json.load(f)

In [12]:
from group_sae import TopKSAE, TopKSAEConfig

sae = TopKSAE(TopKSAEConfig(d_in=cfg["d_in"], c=cfg["expansion_factor"], k=cfg["k"]))

In [14]:
new_state_dict = {
    "W_enc": state_dict.get_tensor("encoder.weight").T,
    "b_enc": state_dict.get_tensor("encoder.bias"),
    "W_dec": state_dict.get_tensor("W_dec"),
    "b_dec": state_dict.get_tensor("b_dec"),
}

sae.load_state_dict(state_dict=new_state_dict)

<All keys matched successfully>

['W_dec', 'b_dec', 'encoder.bias', 'encoder.weight', 'log_threshold']